In [1]:
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from collections import Counter
import gensim.downloader as api
import numpy as np
import fasttext
import fasttext.util
import json

/home/bach/anaconda3/envs/ntu_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']


In [4]:
nltk.download('punkt')

# Tokenize and build the vocabulary from the training dataset
vocab = Counter()

# Tokenize and update vocabulary
for item in train_dataset:
    tokens = word_tokenize(item['text'].lower())  # Tokenizing and converting to lowercase
    vocab.update(tokens)

# Get the size of the vocabulary
vocab_size = len(vocab)
print(f"Vocabulary Size: {vocab_size}")

# Display the top 10 most common words in the dataset (can delete this, for fun :)
print("Most common words in the dataset:", vocab.most_common(10))

# Load pre-trained Word2Vec model
word2vec_model = api.load('word2vec-google-news-300')

[nltk_data] Downloading package punkt to /home/bach/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vocabulary Size: 18029
Most common words in the dataset: [('.', 11197), ('the', 8034), (',', 8001), ('a', 5879), ('and', 4918), ('of', 4815), ('to', 3416), ("'s", 2872), ('is', 2848), ('it', 2733)]


In [4]:
oov_words_word2vec = [word for word in vocab if word not in word2vec_model]
oov_count_word2vec = len(oov_words_word2vec)

print(f"Number of OOV words: {oov_count_word2vec}")
print("Some OOV words:", oov_words_word2vec[:10])

Number of OOV words: 3612
Some OOV words: ['to', '21st', "'s", '``', 'and', 'a', ',', 'jean-claud', 'damme', 'segal']


In [5]:
w2v_vocab = word2vec_model.key_to_index

In [6]:
word2vec_model.get_vector('UNK').shape

(300,)

In [18]:
w2v_vocab['<s>']

: 

In [9]:
with open('w2v_vocab.json', 'w') as f:
    json.dump(w2v_vocab, f)

In [8]:
np.save('w2v_embedding.npy', word2vec_model.vectors)

In [10]:
fasttext_model = fasttext.load_model('../cc.en.300.bin')

embedding_dim = 300 # match input dimension of layer, dimension 300 because fasttext model default is 300 dimensions
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))

word_index = {word: idx + 1 for idx, (word, _) in enumerate(vocab.items())}

In [11]:
word_index

{'the': 1,
 'rock': 2,
 'is': 3,
 'destined': 4,
 'to': 5,
 'be': 6,
 '21st': 7,
 'century': 8,
 "'s": 9,
 'new': 10,
 '``': 11,
 'conan': 12,
 'and': 13,
 'that': 14,
 'he': 15,
 'going': 16,
 'make': 17,
 'a': 18,
 'splash': 19,
 'even': 20,
 'greater': 21,
 'than': 22,
 'arnold': 23,
 'schwarzenegger': 24,
 ',': 25,
 'jean-claud': 26,
 'van': 27,
 'damme': 28,
 'or': 29,
 'steven': 30,
 'segal': 31,
 '.': 32,
 'gorgeously': 33,
 'elaborate': 34,
 'continuation': 35,
 'of': 36,
 'lord': 37,
 'rings': 38,
 'trilogy': 39,
 'so': 40,
 'huge': 41,
 'column': 42,
 'words': 43,
 'can': 44,
 'not': 45,
 'adequately': 46,
 'describe': 47,
 'co-writer/director': 48,
 'peter': 49,
 'jackson': 50,
 'expanded': 51,
 'vision': 52,
 'j': 53,
 'r': 54,
 'tolkien': 55,
 'middle-earth': 56,
 'effective': 57,
 'but': 58,
 'too-tepid': 59,
 'biopic': 60,
 'if': 61,
 'you': 62,
 'sometimes': 63,
 'like': 64,
 'go': 65,
 'movies': 66,
 'have': 67,
 'fun': 68,
 'wasabi': 69,
 'good': 70,
 'place': 71,
 's

In [39]:
fasttext_model = fasttext.load_model('../cc.en.300.bin')

embedding_dim = 300 # match input dimension of layer, dimension 300 because fasttext model default is 300 dimensions
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))

word_index = {word: idx + 1 for idx, (word, _) in enumerate(vocab.items())}

with open('vocab.json', 'w') as f:
    json.dump(word_index, f)
print(word_index)
for word, i in word_index.items():
    if word in word2vec_model:
        # If word is in Word2Vec, use its vector
        embedding_matrix[i] = word2vec_model[word]
    else:
        #embedding_matrix[i] = np.random.normal(size=(embedding_dim,))  # Random for OOV words
        # If OOV, use FastText to generate embedding
        embedding_matrix[i] = fasttext_model.get_word_vector(word)
embedding_matrix = np.array(embedding_matrix, dtype=np.float32)
print(embedding_matrix)
np.save('embedded_matrix.npy', embedding_matrix)


KeyboardInterrupt: 